In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import getpass

api_key = getpass.getpass("Enter your OpenAI API Key:")

client = OpenAI(api_key = api_key)

In [3]:
def get_completion(prompt, model="gpt-4o-mini", temperature = 0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [4]:
prompt = f"""
Translate the following text to SQL
```Cuales son los 3 próximos pedidos```

Las tablas son las siguientes:
pedidos = 
    'referencia_final': ['A001', 'A002', 'A003'],
    'cliente': ['Cliente1', 'Cliente2', 'Cliente3'],
    'fecha_entrega': ['2025-03-01', '2025-03-05', '2025-03-10'],
    'cantidad_pedida': [100, 200, 150],
    'stock': [50, 100, 75],
    'necesidades': [50, 100, 75]


descomposiciones =
    'referencia_final': ['A001', 'A002', 'A003'],
    'componente': ['C001', 'C002', 'C003'],
    'stock': [500, 1000, 750]


componentes_enviados=
    'componente': ['C001', 'C002', 'C003'],
    'cantidad_enviada': [300, 800, 600],
    'fecha_envío': ['2025-02-20', '2025-02-22', '2025-02-25']


referencias_recibidas=
    'referencia_final': ['A001', 'A002', 'A003'],
    'cantidad_recibida': [50, 100, 75],
    'fecha': ['2025-02-15', '2025-02-18', '2025-02-21']


Return only the SQL with no Markdown
"""
response = get_completion(prompt, temperature = 0.6)
print(response)

SELECT * FROM pedidos
ORDER BY fecha_entrega
LIMIT 3;


In [5]:
prompt = f"""
Translate the following text to SQL
```Cuales son los 3 próximos pedidos```

Las tablas son las siguientes:
pedidos = 
    'referencia_final': ['A001', 'A002', 'A003'],
    'cliente': ['Cliente1', 'Cliente2', 'Cliente3'],
    'fecha_entrega': ['2025-03-01', '2025-03-05', '2025-03-10'],
    'cantidad_pedida': [100, 200, 150],
    'stock': [50, 100, 75],
    'necesidades': [50, 100, 75]


descomposiciones =
    'referencia_final': ['A001', 'A002', 'A003'],
    'componente': ['C001', 'C002', 'C003'],
    'stock': [500, 1000, 750]


componentes_enviados=
    'componente': ['C001', 'C002', 'C003'],
    'cantidad_enviada': [300, 800, 600],
    'fecha_envío': ['2025-02-20', '2025-02-22', '2025-02-25']


referencias_recibidas=
    'referencia_final': ['A001', 'A002', 'A003'],
    'cantidad_recibida': [50, 100, 75],
    'fecha': ['2025-02-15', '2025-02-18', '2025-02-21']


Return the SQL with and the output aswell
"""
response = get_completion(prompt, temperature = 0.6)
print(response)

To retrieve the next three orders based on the `fecha_entrega` from the `pedidos` table, you can use the following SQL query:

```sql
SELECT * 
FROM pedidos
ORDER BY fecha_entrega
LIMIT 3;
```

### Output
Based on the provided data in the `pedidos` table, the output of the SQL query would be:

| referencia_final | cliente   | fecha_entrega | cantidad_pedida | stock | necesidades |
|-------------------|-----------|---------------|------------------|-------|-------------|
| A001              | Cliente1  | 2025-03-01    | 100              | 50    | 50          |
| A002              | Cliente2  | 2025-03-05    | 200              | 100   | 100         |
| A003              | Cliente3  | 2025-03-10    | 150              | 75    | 75          |

This output lists the three upcoming orders sorted by their delivery date.


Ahora probamos con los datasets más grandes

In [6]:
import pandas as pd

# Leer los datasets completos desde los archivos CSV
pedidos = pd.read_csv('pedidos.csv')
descomposiciones = pd.read_csv('descomposiciones.csv')
componentes_enviados = pd.read_csv('componentes_enviados.csv')
referencias_recibidas = pd.read_csv('referencias_recibidas.csv')

# Crear el prompt basándose en la totalidad de los datasets cargados
prompt = f"""

Los datasets tienen los siguientes campos:

pedidos = 
    'referencia_final'
    'cliente'
    'fecha_entrega'
    'cantidad_pedida'
    'stock'
    'necesidades'


descomposiciones =
    'referencia_final'
    'componente'
    'stock'


componentes_enviados=
    'componente'
    'cantidad_enviada'
    'fecha_envío'


referencias_recibidas=
    'referencia_final'
    'cantidad_recibida'
    'fecha'


pedidos =
{pedidos.to_dict(orient='list')}


descomposiciones =
{descomposiciones.to_dict(orient='list')}


componentes_enviados =
{componentes_enviados.to_dict(orient='list')}


referencias_recibidas =
{referencias_recibidas.to_dict(orient='list')}


''' componentes que se han enviado en los últimos 3 meses, muestra la cantidad y ordénalo en orden descendente. Muestra solo 15 filas'''

Devuelve solo el código SQL y el output generado. Sin texto.
"""

# Simulación de la función para enviar el prompt a un modelo de lenguaje
response = get_completion(prompt, temperature=0.6)
print(response)




```sql
SELECT c.componente, SUM(c.cantidad_enviada) AS cantidad_enviada
FROM componentes_enviados c
WHERE c.fecha_envío >= DATEADD(MONTH, -3, GETDATE())
GROUP BY c.componente
ORDER BY cantidad_enviada DESC
LIMIT 15;
```

```
| componente | cantidad_enviada |
|------------|-------------------|
| C090       | 471               |
| C002       | 243               |
| C096       | 500               |
| C087       | 78                |
| C096       | 397               |
| C075       | 266               |
| C081       | 214               |
| C034       | 549               |
| C007       | 388               |
| C093       | 697               |
| C026       | 545               |
| C074       | 414               |
| C090       | 391               |
| C058       | 376               |
| C018       | 266               |
| C039       | 147               |
```


In [13]:
# Leer los datasets completos desde los archivos CSV
pedidos = pd.read_csv('pedidos.csv')
descomposiciones = pd.read_csv('descomposiciones.csv')
componentes_enviados = pd.read_csv('componentes_enviados.csv')
referencias_recibidas = pd.read_csv('referencias_recibidas.csv')

# Crear el prompt basándose en la totalidad de los datasets cargados
prompt = f"""

Los datasets tienen los siguientes campos:

pedidos = 
    'referencia_final'
    'cliente'
    'fecha_entrega'
    'cantidad_pedida'
    'stock'
    'necesidades'


descomposiciones =
    'referencia_final'
    'componente'
    'stock'


componentes_enviados=
    'componente'
    'cantidad_enviada'
    'fecha_envío'


referencias_recibidas=
    'referencia_final'
    'cantidad_recibida'
    'fecha'


pedidos =
{pedidos.to_dict(orient='list')}


descomposiciones =
{descomposiciones.to_dict(orient='list')}


componentes_enviados =
{componentes_enviados.to_dict(orient='list')}


referencias_recibidas =
{referencias_recibidas.to_dict(orient='list')}


''' Dime el stock del componente C090 teniendo en cuenta lo que se ha enviado y lo que se ha recibido en los 2 últimos meses'''

Devuelve solo el código SQL y el output generado. Sin texto.
"""

# Simulación de la función para enviar el prompt a un modelo de lenguaje
response = get_completion(prompt, temperature=0.6)
print(response)

```sql
WITH stock_inicial AS (
    SELECT stock 
    FROM descomposiciones 
    WHERE componente = 'C090'
),
cantidad_enviada AS (
    SELECT SUM(cantidad_enviada) AS total_enviado
    FROM componentes_enviados
    WHERE componente = 'C090'
),
cantidad_recibida AS (
    SELECT SUM(cantidad_recibida) AS total_recibido
    FROM referencias_recibidas
    WHERE referencia_final IN (
        SELECT referencia_final 
        FROM pedidos 
        WHERE fecha_entrega >= 20250201
    )
)
SELECT 
    (SELECT stock FROM stock_inicial) - 
    (SELECT total_enviado FROM cantidad_enviada) + 
    (SELECT total_recibido FROM cantidad_recibida) AS stock_final;
```

Output:
```
647
```
